In [4]:
import glob
import pretty_midi
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers
import os
from keras_self_attention import SeqSelfAttention


## Transform MIDI files to PrettyMIDI


In [1]:
# targetPattern = r"../raw_data/*/*/*/*/*.mid"

# midi_files_paths = glob.glob(targetPattern)

# for file in midi_files_paths:
#     try:
#         joblib.dump(pretty_midi.PrettyMIDI(file), f'../raw_data/pretty_midi/{file.split("/")[-1][:-4]}')
        
        
#     except:
        
        
#         pass

## Visualize PrettyMIDI files


### Test on 1 random file


In [2]:
%%capture --no-display

from visual_midi import Plotter
from visual_midi import Preset
from pretty_midi import PrettyMIDI

pm = joblib.load('../raw_data/pretty_midi/Addicted')

preset = Preset(plot_width=850)
plotter = Plotter(preset) #, plot_max_length_bar=4

plotter.show_notebook(pm)

Loading BokehJS ...

Column(id='1535', ...)

In [3]:
pm_piano = pm.get_piano_roll()

In [4]:
joblib.dump(pm, 'test')

['test']

In [5]:
test = joblib.load('test')

In [6]:
type(test)

pretty_midi.pretty_midi.PrettyMIDI

In [7]:
%%capture --no-display

plotter.show_notebook(test)

Loading BokehJS ...

Column(id='2114', ...)

### Get PianoRoll to Pretty Midi function from Utils


In [10]:
os.chdir('/home/dianehb/code/KarlVoncq/MIDIComposingAI')

In [3]:
os.getcwd()

'/home/dianehb/code/KarlVoncq/MIDIComposingAI'

In [16]:
from MIDIComposingAI.utils import piano_roll_to_pretty_midi

## Test Guitar Chords DataSet


### Get Dataset



In [47]:
guitar_chords_df = pd.read_csv('raw_data/guitar-chords-midi-pitches.csv', sep=";")

In [71]:
guitar_chords_df.head()

,Chord,2nd,3rd,4th,5th,6th
0,Am,45,52,57.0,60.0,64.0
1,Amaj,45,52,57.0,61.0,64.0
2,A6,52,57,61.0,66.0,NaN
3,A7,45,52,55.0,61.0,64.0
4,A7b5,45,55,61.0,63.0,NaN


In [49]:
columns = guitar_chords_df.columns

In [50]:
columns_to_transform = columns[1:]

In [51]:
columns_to_transform # Can't be transformed to integer yet because of NaN values

Index(['2nd', '3rd', '4th', '5th', '6th'], dtype='object')

In [66]:
# NEED to DROP NA if the chords we use are shorter than 5th line.dropna()
# In model user mask & padding with -1 for example ?

### Create MIDI test files


#### Create MIDI with first line





In [70]:
guitar_chord_1_test = guitar_chords_df.iloc[0]

In [37]:
guitar_chord_1 = guitar_chord_1_test.values[1:]

In [56]:
# Create a PrettyMIDI object
cello_c_chord = pretty_midi.PrettyMIDI()

# Create an Instrument instance for a cello instrument

cello_program = pretty_midi.instrument_name_to_program('Cello')


cello = pretty_midi.Instrument(program=cello_program)

# Iterate over notes
for note_midi in guitar_chord_1:
    
    # Create a Note instance, starting at 0s and ending at .5s
    note = pretty_midi.Note(velocity=100, pitch=int(note_midi), start=0, end=.5)
    
    # Add it to our cello instrument
    
    cello.notes.append(note)
    
# Add the cello instrument to the PrettyMIDI object

cello_c_chord.instruments.append(cello)

# Write out the MIDI data
cello_c_chord.write('raw_data/guitar-test-chord.mid')

#### Create MIDI with last line



In [57]:
guitar_chord_2_test = guitar_chords_df.iloc[-1]

In [58]:
guitar_chord_2 = guitar_chord_2_test.values[1:]

In [59]:
# TEST 2

cello_c_chord = pretty_midi.PrettyMIDI()
cello_program = pretty_midi.instrument_name_to_program('Cello')
cello = pretty_midi.Instrument(program=cello_program)

for note_midi in guitar_chord_2:
    
    note = pretty_midi.Note(velocity=100, pitch=int(note_midi), start=0, end=.5)
    cello.notes.append(note)

cello_c_chord.instruments.append(cello)

# Write out the MIDI data
cello_c_chord.write('raw_data/guitar-test2-chord.mid')

## Test Music Scale Dataset


### Get Dataset


In [74]:
music_scale_df = pd.read_csv('raw_data/music-scales_with_columns.csv', sep=";")

In [75]:
music_scale_df.head()

,scale_name,root,steps_from_root,steps_from_root.1,steps_from_root.2,steps_from_root.3,steps_from_root.4,steps_from_root.5,steps_from_root.6,steps_from_root.7,steps_from_root.8,other
0,ChromaticTriMirror,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DoReMi,0,2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Flat6and7,0,10,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MajorFlat6,0,4,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MajorTriad1,0,3,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Get Samples & Fix Features/Target




### Concat Samples into one


In [61]:
df0 = joblib.load(f'raw_data/dataframes_sample/dataframe_0')

In [32]:
data = pd.DataFrame(columns=['accompaniment', 'melody_pitches', 'melody_velocities'])
for i in range(10):
    loaded = joblib.load(f'raw_data/dataframes_sample/dataframe_{i}')
    data = pd.concat((data, loaded))

In [33]:
data.reset_index(drop=True, inplace=True)

In [56]:
acc = [np.asarray(data['accompaniment'].iloc[0].todense()).T, np.asarray(data['accompaniment'].iloc[1].todense()).T]
pitch = [np.asarray(data['melody_pitches'].iloc[0].todense()).reshape(-1), np.asarray(data['melody_pitches'].iloc[1].todense()).reshape(-1)]
velo = [np.asarray(data['melody_velocities'].iloc[0].todense()).reshape(-1), np.asarray(data['melody_velocities'].iloc[0].todense()).reshape(-1)]

In [57]:
acc_as_array = np.array(acc)
pitch_as_array = np.array(pitch)
velo_as_array = np.array(velo)

In [59]:
acc_as_array.shape
# n_seqs, n_observations, n_features
# Pour garder le voc des cours ML n_obs, n_indiv, n_features

(2, 10000, 128)

In [60]:
pitch_as_array.shape

(2, 10000)

In [37]:
X_dumb_train = acc_as_array
y_pitch_dumb_train = pitch_as_array
y_velocity_dumb_train = velo_as_array

In [23]:
for column in data:
    for i in range(len(data[column])):
        data[column][i] = np.asarray(data[column][i].todense())

### Split Features & Target + Train Test Split


In [11]:
X, y_pitch, y_velocity = joblib.load(f'raw_data/nparray_samples/nparray0') # No .. because of os change dir earlier

for i in range(1, 10):
    loaded = joblib.load(f'raw_data/nparray_samples/nparray{i}')
    
    X = np.concatenate((X, loaded[0]))
    y_pitch = np.concatenate((y_pitch, loaded[1]))
    y_velocity = np.concatenate((y_velocity, loaded[2]))

In [12]:
X_train, X_test, y_train_pitch, y_test_pitch, y_train_velocity, y_test_velocity  = train_test_split(X, y_pitch, y_velocity, test_size=0.3)

## Models Tests



### Test Model from Article

In [ ]:
# https://github.com/CyberZHG/keras-self-attention/blob/master/keras_self_attention/seq_self_attention.py
class SeqSelfAttention(tf.keras.layers.Layer):

    ATTENTION_TYPE_ADD = 'additive'
    ATTENTION_TYPE_MUL = 'multiplicative'

    def __init__(self,
                 units=32,
                 attention_width=None,
                 attention_type=ATTENTION_TYPE_ADD,
                 return_attention=False,
                 history_only=False,
                 kernel_initializer='glorot_normal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 use_additive_bias=True,
                 use_attention_bias=True,
                 attention_activation=None,
                 attention_regularizer_weight=0.0,
                 **kwargs):
        """Layer initialization.
        For additive attention, see: https://arxiv.org/pdf/1806.01264.pdf
        :param units: The dimension of the vectors that used to calculate the attention weights.
        :param attention_width: The width of local attention.
        :param attention_type: 'additive' or 'multiplicative'.
        :param return_attention: Whether to return the attention weights for visualization.
        :param history_only: Only use historical pieces of data.
        :param kernel_initializer: The initializer for weight matrices.
        :param bias_initializer: The initializer for biases.
        :param kernel_regularizer: The regularization for weight matrices.
        :param bias_regularizer: The regularization for biases.
        :param kernel_constraint: The constraint for weight matrices.
        :param bias_constraint: The constraint for biases.
        :param use_additive_bias: Whether to use bias while calculating the relevance of inputs features
                                  in additive mode.
        :param use_attention_bias: Whether to use bias while calculating the weights of attention.
        :param attention_activation: The activation used for calculating the weights of attention.
        :param attention_regularizer_weight: The weights of attention regularizer.
        :param kwargs: Parameters for parent class.
        """
        self.supports_masking = True
        self.units = units
        self.attention_width = attention_width
        self.attention_type = attention_type
        self.return_attention = return_attention
        self.history_only = history_only
        if history_only and attention_width is None:
            self.attention_width = int(1e9)

        self.use_additive_bias = use_additive_bias
        self.use_attention_bias = use_attention_bias
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
        self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
        self.bias_constraint = tf.keras.constraints.get(bias_constraint)
        self.attention_activation = tf.keras.activations.get(attention_activation)
        self.attention_regularizer_weight = attention_regularizer_weight
        self._backend = tf.keras.backend.backend()

        if attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self.Wx, self.Wt, self.bh = None, None, None
            self.Wa, self.ba = None, None
        elif attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self.Wa, self.ba = None, None
        else:
            raise NotImplementedError('No implementation for attention type : ' + attention_type)

        super(SeqSelfAttention, self).__init__(**kwargs)
    def get_config(self):
        config = {
            'units': self.units,
            'attention_width': self.attention_width,
            'attention_type': self.attention_type,
            'return_attention': self.return_attention,
            'history_only': self.history_only,
            'use_additive_bias': self.use_additive_bias,
            'use_attention_bias': self.use_attention_bias,
            'kernel_initializer': tf.keras.regularizers.serialize(self.kernel_initializer),
            'bias_initializer': tf.keras.regularizers.serialize(self.bias_initializer),
            'kernel_regularizer': tf.keras.regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': tf.keras.regularizers.serialize(self.bias_regularizer),
            'kernel_constraint': tf.keras.constraints.serialize(self.kernel_constraint),
            'bias_constraint': tf.keras.constraints.serialize(self.bias_constraint),
            'attention_activation': tf.keras.activations.serialize(self.attention_activation),
            'attention_regularizer_weight': self.attention_regularizer_weight,
        }
        base_config = super(SeqSelfAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]
        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self._build_additive_attention(input_shape)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self._build_multiplicative_attention(input_shape)
        super(SeqSelfAttention, self).build(input_shape)
        
    def _build_additive_attention(self, input_shape):
        feature_dim = input_shape[2]

        self.Wt = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wt'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        self.Wx = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wx'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_additive_bias:
            self.bh = self.add_weight(shape=(self.units,),
                                      name='{}_Add_bh'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

        self.Wa = self.add_weight(shape=(self.units, 1),
                                  name='{}_Add_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Add_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)
    def _build_multiplicative_attention(self, input_shape):
        feature_dim = input_shape[2]

        self.Wa = self.add_weight(shape=(feature_dim, feature_dim),
                                  name='{}_Mul_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Mul_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)
    def call(self, inputs, mask=None, **kwargs):
        if isinstance(inputs, list):
            inputs, positions = inputs
            positions = K.cast(positions, 'int32')
            mask = mask[1]
        else:
            positions = None

        input_len = K.shape(inputs)[1]

        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            e = self._call_additive_emission(inputs)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            e = self._call_multiplicative_emission(inputs)

        if self.attention_activation is not None:
            e = self.attention_activation(e)
        e = K.exp(e - K.max(e, axis=-1, keepdims=True))
        if self.attention_width is not None:
            ones = tf.ones((input_len, input_len))
            if self.history_only:
                local = tf.linalg.band_part(
                    ones,
                    K.minimum(input_len, self.attention_width - 1),
                    0,
                )
            else:
                local = tf.linalg.band_part(
                    ones,
                    K.minimum(input_len, self.attention_width // 2),
                    K.minimum(input_len, (self.attention_width - 1) // 2),
                )
            e = e * K.expand_dims(local, 0)
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            e = K.permute_dimensions(K.permute_dimensions(e * mask, (0, 2, 1)) * mask, (0, 2, 1))
                # a_{t} = \text{softmax}(e_t)
        s = K.sum(e, axis=-1)
        s = K.tile(K.expand_dims(s, axis=-1), K.stack([1, 1, input_len]))
        a = e / (s + K.epsilon())

        # l_t = \sum_{t'} a_{t, t'} x_{t'}
        v = K.batch_dot(a, inputs)
        if self.attention_regularizer_weight > 0.0:
            self.add_loss(self._attention_regularizer(a))

        if positions is not None:
            pos_num = K.shape(positions)[1]
            batch_indices = K.tile(K.expand_dims(K.arange(K.shape(inputs)[0]), axis=-1), K.stack([1, pos_num]))
            pos_indices = K.stack([batch_indices, positions], axis=-1)
            v = tf.gather_nd(v, pos_indices)
            a = tf.gather_nd(a, pos_indices)

        if self.return_attention:
            return [v, a]
        return v
    def _call_additive_emission(self, inputs):
        input_shape = K.shape(inputs)
        batch_size, input_len = input_shape[0], input_shape[1]

        # h_{t, t'} = \tanh(x_t^T W_t + x_{t'}^T W_x + b_h)
        q, k = K.dot(inputs, self.Wt), K.dot(inputs, self.Wx)
        q = K.tile(K.expand_dims(q, 2), K.stack([1, 1, input_len, 1]))
        k = K.tile(K.expand_dims(k, 1), K.stack([1, input_len, 1, 1]))
        if self.use_additive_bias:
            h = K.tanh(q + k + self.bh)
        else:
            h = K.tanh(q + k)

        # e_{t, t'} = W_a h_{t, t'} + b_a
        if self.use_attention_bias:
            e = K.reshape(K.dot(h, self.Wa) + self.ba, (batch_size, input_len, input_len))
        else:
            e = K.reshape(K.dot(h, self.Wa), (batch_size, input_len, input_len))
        return e

    def _call_multiplicative_emission(self, inputs):
        # e_{t, t'} = x_t^T W_a x_{t'} + b_a
        e = K.batch_dot(K.dot(inputs, self.Wa), K.permute_dimensions(inputs, (0, 2, 1)))
        if self.use_attention_bias:
            e = e + self.ba
        return e
    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape, pos_shape = input_shape
            output_shape = (input_shape[0], pos_shape[1], input_shape[2])
        else:
            output_shape = input_shape
        if self.return_attention:
            attention_shape = (input_shape[0], output_shape[1], input_shape[1])
            return [output_shape, attention_shape]
        return output_shape

    def compute_mask(self, inputs, mask=None):
        if isinstance(inputs, list):
            mask = mask[1]
        if self.return_attention:
            return [mask, None]
        return mask

    def _attention_regularizer(self, attention):
        batch_size = K.cast(K.shape(attention)[0], K.floatx())
        input_len = K.shape(attention)[-1]
        return self.attention_regularizer_weight * K.sum(K.square(K.batch_dot(
            attention,
            K.permute_dimensions(attention, (0, 2, 1))) - tf.eye(input_len))) / batch_size

    @staticmethod
    def get_custom_objects():
      return {'SeqSelfAttention': SeqSelfAttention}

In [ ]:
def create_model(seq_len, unique_notes, dropout=0.3, output_emb=100, rnn_unit=128, dense_unit=64):
  inputs = tf.keras.layers.Input(shape=(seq_len,))
  embedding = tf.keras.layers.Embedding(input_dim=unique_notes+1, output_dim=output_emb, input_length=seq_len)(inputs)
  forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(embedding)
  forward_pass , att_vector = SeqSelfAttention(
      return_attention=True,
      attention_activation='sigmoid', 
      attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
      attention_width=50, 
      kernel_regularizer=tf.keras.regularizers.l2(1e-4),
      bias_regularizer=tf.keras.regularizers.l1(1e-4),
      attention_regularizer_weight=1e-4,
  )(forward_pass)
  forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
  forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit, return_sequences=True))(forward_pass)
  forward_pass , att_vector2 = SeqSelfAttention(
      return_attention=True,
      attention_activation='sigmoid', 
      attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
      attention_width=50, 
      kernel_regularizer=tf.keras.regularizers.l2(1e-4),
      bias_regularizer=tf.keras.regularizers.l1(1e-4),
      attention_regularizer_weight=1e-4,
  )(forward_pass)
  forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
  forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit))(forward_pass)
  forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
  forward_pass = tf.keras.layers.Dense(dense_unit)(forward_pass)
  forward_pass = tf.keras.layers.LeakyReLU()(forward_pass)
  outputs = tf.keras.layers.Dense(unique_notes+1, activation = "softmax")(forward_pass)

  model = tf.keras.Model(inputs=inputs, outputs=outputs, name='generate_scores_rnn')
  return model

model = create_model(seq_len, unique_notes)

### Simple Model with LSTM



#### LSTM - Classique


##### LSTM - Classique - Regression



###### Model Creation




In [24]:
inputs = layers.Input(shape=(500, 128), name='main_input') # TO BE MODIFIED - SPARSE MATRIX  #input shape 128 * 10_000
# shape = X_train[0].shape
# Envoi d'un sparse tensor sinon sparse matrix
# Envoyer en input sparse ou brut est-ce que ça change les perfs ou non
# https://stackoverflow.com/questions/20459536/convert-pandas-dataframe-to-sparse-numpy-matrix-directly



main_branch = layers.LSTM(units=128, activation='tanh', return_sequences=True)(inputs)
main_branch = layers.LSTM(units=128, activation='tanh')(main_branch)
main_branch = layers.Dense(64, activation='relu')(main_branch)
# DO WE NEED TWO LAYERS FOR REGRESSION AS WELL ?
pitch_branch = layers.Dense(500, activation='relu', name='pitch_output')(main_branch)
velocity_branch = layers.Dense(500, activation='relu', name='velocity_output')(main_branch)


model_lstm_reg = models.Model(inputs = inputs, outputs = [pitch_branch, velocity_branch])

###### Model Summary



In [25]:
model_lstm_reg.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 500, 128)]   0           []                               
                                                                                                  
 lstm_12 (LSTM)                 (None, 500, 128)     131584      ['main_input[0][0]']             
                                                                                                  
 lstm_13 (LSTM)                 (None, 128)          131584      ['lstm_12[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 64)           8256        ['lstm_13[0][0]']                
                                                                                            

###### Model Compilation




In [26]:
model_lstm_reg.compile(loss='mse', optimizer='rmsprop') # LOSS TO REVIEW


In [27]:
model_lstm_reg.fit(X_train,
              {'pitch_output': y_train_pitch, 'velocity_output': y_train_pitch},   
              epochs=3,    
              verbose=1,    
                   
             )

Epoch 1/3
2/2 [==============================] - 5s 819ms/step - loss: 6843.3013 - pitch_output_loss: 3421.5637 - velocity_output_loss: 3421.7375
Epoch 2/3
2/2 [==============================] - 2s 818ms/step - loss: 6812.8818 - pitch_output_loss: 3405.8647 - velocity_output_loss: 3407.0171
Epoch 3/3
2/2 [==============================] - 2s 816ms/step - loss: 6709.1758 - pitch_output_loss: 3352.1475 - velocity_output_loss: 3357.0283


In [19]:
X_test.shape


(24, 500, 128)

In [28]:
predictions = model_lstm_reg.predict(X_test)


In [29]:
len(predictions)


2

In [30]:
predictions[0].shape


(24, 500)

In [31]:
predictions[0]

array([[1.7282021 , 0.81933546, 1.5780133 , ..., 2.1836374 , 1.81944   ,
        0.7153228 ],
       [1.6985056 , 0.89310086, 1.5660357 , ..., 2.1188452 , 1.7540379 ,
        0.72517514],
       [1.774209  , 0.8511097 , 1.603918  , ..., 2.2873428 , 1.8960465 ,
        0.7071736 ],
       ...,
       [1.7526388 , 0.74925786, 1.6756169 , ..., 2.5557847 , 1.7794411 ,
        0.86734444],
       [1.6046493 , 0.7806285 , 1.4707662 , ..., 2.062016  , 1.7441509 ,
        0.6406468 ],
       [1.7776222 , 0.85299087, 1.6044803 , ..., 2.2847514 , 1.8972899 ,
        0.7062103 ]], dtype=float32)

In [3]:
# https://towardsdatascience.com/zero-inflated-regression-c7dfc656d8af


##### LSTM - Classique - Classification


###### Model Creation


In [13]:
X_train[0].shape

(500, 128)

In [14]:
inputs = layers.Input(shape=(500, 128), name='main_input') # TO BE MODIFIED - SPARSE MATRIX

main_branch = layers.LSTM(units=128, activation='tanh', return_sequences=True)(inputs)
main_branch = layers.LSTM(units=128, activation='tanh')(main_branch)
main_branch = layers.Dense(64, activation='relu')(main_branch)
               
# 2 SORTIES POUR CHAQUE PREDICTION
# Multitask learning
# https://github.com/rahul-pande/faces-mtl
# https://github.com/rahul-pande/faces-mtl/blob/master/faces_mtl_age_gender.ipynb

pitch_branch = layers.Dense(500, activation='softmax', name='pitch_output')(main_branch)
velocity_branch = layers.Dense(500, activation='softmax', name='velocity_output')(main_branch)



# Cannot use Sequential because several outputs
model_lstm_class = models.Model(inputs = inputs, outputs = [pitch_branch, velocity_branch])

###### Model Summary


In [15]:
model_lstm_class.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 500, 128)]   0           []                               
                                                                                                  
 lstm_8 (LSTM)                  (None, 500, 128)     131584      ['main_input[0][0]']             
                                                                                                  
 lstm_9 (LSTM)                  (None, 128)          131584      ['lstm_8[0][0]']                 
                                                                                                  
 dense_2 (Dense)                (None, 64)           8256        ['lstm_9[0][0]']                 
                                                                                            

###### Model Compilation


In [16]:
model_lstm_class.compile(loss='mse', optimizer='rmsprop') # LOSS TO REVIEW



In [17]:
model_lstm_class.fit(X_train,
                     {'pitch_output': y_train_pitch, 'velocity_output': y_train_pitch},
                     epochs=3,
                     verbose=1,
                     validation_split=0.3)

Epoch 1/3
2/2 [==============================] - 6s 1s/step - loss: 7334.0229 - pitch_output_loss: 3667.0115 - velocity_output_loss: 3667.0115 - val_loss: 5740.1104 - val_pitch_output_loss: 2870.0552 - val_velocity_output_loss: 2870.0554
Epoch 2/3
2/2 [==============================] - 2s 769ms/step - loss: 7334.0210 - pitch_output_loss: 3667.0103 - velocity_output_loss: 3667.0105 - val_loss: 5740.1104 - val_pitch_output_loss: 2870.0552 - val_velocity_output_loss: 2870.0552
Epoch 3/3
2/2 [==============================] - 2s 757ms/step - loss: 7334.0195 - pitch_output_loss: 3667.0093 - velocity_output_loss: 3667.0098 - val_loss: 5740.1104 - val_pitch_output_loss: 2870.0552 - val_velocity_output_loss: 2870.0552


In [18]:
model_lstm_class.predict(X_test)


[array([[0.00220615, 0.00244655, 0.00184569, ..., 0.00191783, 0.00205974,
         0.00223915],
        [0.00225659, 0.0021515 , 0.00203297, ..., 0.00143521, 0.00202373,
         0.00283183],
        [0.00204619, 0.00208152, 0.0020491 , ..., 0.0020014 , 0.00191964,
         0.00204967],
        ...,
        [0.00176206, 0.00220666, 0.00186596, ..., 0.00255523, 0.00295224,
         0.0024419 ],
        [0.002087  , 0.00194584, 0.00214329, ..., 0.00213238, 0.00197001,
         0.00227359],
        [0.00204496, 0.00207931, 0.0020483 , ..., 0.00200137, 0.00192178,
         0.00204961]], dtype=float32),
 array([[0.00213919, 0.00188687, 0.00198557, ..., 0.00183318, 0.00219967,
         0.00208585],
        [0.00294387, 0.00221825, 0.00226866, ..., 0.00154194, 0.00215878,
         0.00221682],
        [0.00195877, 0.00197917, 0.00195863, ..., 0.00200432, 0.00199127,
         0.001916  ],
        ...,
        [0.00268113, 0.00208994, 0.00111118, ..., 0.00183242, 0.00245036,
         0.00192286

#### LSTM - Bidirectional


##### LSTM - Bidirectional - Regression



###### Model Creation




In [ ]:
# https://keras.io/examples/nlp/bidirectional_lstm_imdb/
# https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
# Try all methods concat, sum etc..

In [49]:
X_train.shape



(53, 500, 128)

In [53]:
inputs = layers.Input(shape=(500, 128), name='main_input') # TO BE MODIFIED - SPARSE MATRIX

# Add 2 bidirectional LSTMs
main_branch = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)

main_branch = layers.Bidirectional(layers.LSTM(128, return_sequences=False))(main_branch)


pitch_branch = layers.Dense(500, activation='relu', name='pitch_output')(main_branch)
velocity_branch = layers.Dense(500, activation='relu', name='velocity_output')(main_branch)

model_lstm_bin_reg = models.Model(inputs = inputs, outputs = [pitch_branch, velocity_branch])

###### Model Summary


In [11]:
model_lstm_bin_reg.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 10000, 128)  0           []                               
                                ]                                                                 
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 10000, 256)  263168      ['main_input[0][0]']             
 )                                                                                                
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 10000, 256)  394240      ['bidirectional_1[0][0]']        
 )                                                                                            

###### Model Compilation



In [55]:
model_lstm_bin_reg.compile(loss='mse', optimizer='rmsprop') # LOSS TO REVIEW




In [56]:
model_lstm_bin_reg.fit(X_train,
                       
                       {'pitch_output': y_train_pitch, 'velocity_output': y_train_pitch},
                       epochs=3,
                       
                       verbose=1,
                       validation_split=0.3)




Epoch 1/3
2/2 [==============================] - 9s 2s/step - loss: 7316.5234 - pitch_output_loss: 3658.3159 - velocity_output_loss: 3658.2078 - val_loss: 5709.1035 - val_pitch_output_loss: 2854.4509 - val_velocity_output_loss: 2854.6523
Epoch 2/3
2/2 [==============================] - 3s 1s/step - loss: 7279.8643 - pitch_output_loss: 3639.9282 - velocity_output_loss: 3639.9358 - val_loss: 5612.9116 - val_pitch_output_loss: 2806.0522 - val_velocity_output_loss: 2806.8594
Epoch 3/3
2/2 [==============================] - 3s 1s/step - loss: 7129.2812 - pitch_output_loss: 3564.2854 - velocity_output_loss: 3564.9961 - val_loss: 5366.8047 - val_pitch_output_loss: 2682.1904 - val_velocity_output_loss: 2684.6143


In [57]:
model_lstm_bin_reg.predict(X_test)


[array([[1.49013  , 2.5460432, 1.5468057, ..., 0.       , 2.9921162,
         1.4680847],
        [1.4176242, 2.5383046, 1.606751 , ..., 0.       , 3.1321988,
         1.4478189],
        [1.5379738, 2.5684156, 1.5659815, ..., 0.       , 2.8267174,
         1.4381366],
        ...,
        [1.5755632, 2.6621666, 1.3937517, ..., 0.       , 3.0831265,
         1.4822463],
        [1.4692689, 2.512146 , 1.4996699, ..., 0.       , 2.8341765,
         1.5800787],
        [1.5274748, 2.5598228, 1.4604315, ..., 0.       , 2.8393102,
         1.3934698]], dtype=float32),
 array([[2.0669925, 2.926127 , 2.0572948, ..., 2.660007 , 2.6958578,
         1.2443758],
        [2.054878 , 2.8737903, 2.1641245, ..., 2.6332088, 2.6927924,
         1.3162158],
        [2.0483797, 2.894317 , 2.1711962, ..., 2.734638 , 2.485267 ,
         1.1730914],
        ...,
        [2.2032788, 3.1675272, 2.405532 , ..., 2.877417 , 2.5378315,
         1.3443611],
        [2.1885436, 2.6860294, 2.1007724, ..., 2.5330186,

##### LSTM - Bidirectional - Classification



###### Model Creation


In [ ]:
inputs = layers.Input(shape=(10000, 128), name='main_input') # TO BE MODIFIED - SPARSE MATRIX
# Add 2 bidirectional LSTMs
main_branch = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(main_branch)
main_branch = layers.Bidirectional(layers.LSTM(128))(main_branch)
# Add a classifier
pitch_branch = layers.Dense(128, activation='softmax', name='pitch_output')(main_branch)
velocity_branch = layers.Dense(128, activation='softmax', name='velocity_output')(main_branch)
model_lstm_bin_class = models.Model(inputs = inputs, outputs = [pitch_branch, velocity_branch])
model_lstm_bin_class.summary()

###### Model Compilation




In [ ]:
model_lstm_bin_class.compile(loss='mse', optimizer='rmsprop') # LOSS TO REVIEW

#### Test


In [ ]:
# https://github.com/CyberZHG/keras-self-attention -> Try the basic use

In [32]:
from keras_self_attention import SeqSelfAttention

model_self_test = models.Sequential()
model_self_test.add(layers.Bidirectional(layers.LSTM(units=128, return_sequences=True)))
model_self_test.add(SeqSelfAttention(attention_activation='sigmoid'))
model_self_test.add(layers.Dense(units=64))

In [34]:
inputs = layers.Input(shape=(500, 128), name='main_input') # TO BE MODIFIED - SPARSE MATRIX  #input shape 128 * 10_000
# shape = X_train[0].shape
# Envoi d'un sparse tensor sinon sparse matrix
# Envoyer en input sparse ou brut est-ce que ça change les perfs ou non
# https://stackoverflow.com/questions/20459536/convert-pandas-dataframe-to-sparse-numpy-matrix-directly

main_branch = layers.LSTM(units=128, activation='tanh', return_sequences=True)(inputs)
main_branch = layers.LSTM(units=128, activation='tanh', return_sequences=True)(main_branch)

main_branch = SeqSelfAttention(attention_activation='sigmoid')(main_branch)

main_branch = layers.Dense(64, activation='relu')(main_branch)

pitch_branch = layers.Dense(500, activation='relu', name='pitch_output')(main_branch)
velocity_branch = layers.Dense(500, activation='relu', name='velocity_output')(main_branch)

model_self_test = models.Model(inputs = inputs, outputs = [pitch_branch, velocity_branch])

In [35]:
model_self_test.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 500, 128)]   0           []                               
                                                                                                  
 lstm_15 (LSTM)                 (None, 500, 128)     131584      ['main_input[0][0]']             
                                                                                                  
 lstm_16 (LSTM)                 (None, 500, 128)     131584      ['lstm_15[0][0]']                
                                                                                                  
 seq_self_attention_4 (SeqSelfA  (None, 500, 128)    8257        ['lstm_16[0][0]']                
 ttention)                                                                                  

In [36]:
model_self_test.compile(loss='mse', optimizer='rmsprop') # LOSS TO REVIEW

In [37]:
model_self_test.fit(X_train,
                    
                     {'pitch_output': y_train_pitch, 'velocity_output': y_train_pitch},
                     epochs=3,
                     verbose=1,
                     validation_split=0.3)

Epoch 1/3


2021-12-01 18:17:06.395515: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1024000000 exceeds 10% of free system memory.


InvalidArgumentError:  Incompatible shapes: [32,500] vs. [32,500,500]
	 [[node mean_squared_error_1/SquaredDifference
 (defined at /home/dianehb/.pyenv/versions/3.8.12/envs/MIDIComposingAI/lib/python3.8/site-packages/keras/losses.py:1204)
]] [Op:__inference_train_function_28176]

Errors may have originated from an input operation.
Input Source operations connected to node mean_squared_error_1/SquaredDifference:
In[0] model_4/velocity_output/Relu (defined at /home/dianehb/.pyenv/versions/3.8.12/envs/MIDIComposingAI/lib/python3.8/site-packages/keras/backend.py:4867)	
In[1] mean_squared_error_1/Cast (defined at /home/dianehb/.pyenv/versions/3.8.12/envs/MIDIComposingAI/lib/python3.8/site-packages/keras/losses.py:1203)

Operation defined at: (most recent call last)
>>>   File "/home/dianehb/.pyenv/versions/3.8.12/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/home/dianehb/.pyenv/versions/3.8.12/lib/python3.8/run